## sprint23課題 ゲート付きリカレントニューラルネットワーク
1.この課題の目的  
広く手法全般を学ぶ経験をする   
発展的なRNNの手法を理解する   

【目的としないこと】   
LSTMやGRUの細かい部分の理解

### 説明課題

### LSTMやGRUにはゲートと呼ばれるものがあるがこれにはどのような働きがあるか
・シンプルRNNは、時系列データの長期の依存関係を学習することが苦手であるが、その理由は、誤差逆伝播時において、tanhノードを通るたびにその値がどんどん小さくなる原因によって、長い系列データを扱う場合は深い層となり、勾配消失が起きるためである。LSTMやGRUにあるゲート付きRNNは、勾配消失に有効な手法であり、時系列データの長期的な依存関係を学習することができるようになる。   
・ゲートはデータの流れをコントロールするもので、ゲートには専用の重みがあり、sigmoid関数を使って０.０から1.0までの実数を出力する。LSTMには、inputゲート、forgetゲート、outputゲートの３つのゲート、GRUには、resetゲート、updateゲートの2つのゲートがある。

### SimpleRNN、LSTM、GRUはどのように使い分ければ良いか
勾配消失問題によって、一般的にはSimpleRNNは、10時刻(10単語）程度のデータが限界となることが多い。そのため、比較的短いデータに関しての場合のみ、SimpleRNNを使うことが有効である。LSTMは複雑な構造であるが、GRUはLSTMに比べてシンプルな構造となっており、パラメータが少なく、計算時間を短縮している。したがって、GRUはデータセットのサイズが小さい場合や、モデル設計で繰り返しの試行が必要な場合、特に適していると考えられる。


### StackedRNNを使うことはどういった効果があるか
StackedRNN を使用すると 、RNNレイヤーを時間方向（横方向）ではなく、縦に深く重ねる(層を深く)できる。層を深くすることで表現力の高いモデルを作る事ができる。

### ConvLSTMはどういった場面で使われるか
通常のLSTMはlinearな状態で処理されるので、画像などは位置的な情報が死んでしまう。convLSTMは画像の状態を維持したまま入力するので位置情報が保持される。linearの場合の行列演算はconvolutionに置き換えれられる。これによりLSTMの時間情報、convolutionの位置情報が同時にいかされる。convLSTMは画像が系列データとして入力されていくので、動画処理などに使われると見られる。

## 複数のRecurrentレイヤーを実行（データセット：ロイター）

### RNN

In [0]:
# First, let's define a RNN Cell, as a layer subclass.
import keras
from keras import backend as K

class MinimalRNNCell(keras.layers.Layer):

    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size = units
        super(MinimalRNNCell, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='uniform',
            name='recurrent_kernel')
        self.built = True

    def call(self, inputs, states):
        prev_output = states[0]
        h = K.dot(inputs, self.kernel)
        output = h + K.dot(prev_output, self.recurrent_kernel)
        return output, [output]

# Let's use this cell in a RNN layer:

# cell = MinimalRNNCell(32)
# x = keras.Input((None, 5))
# layer = RNN(cell)
# y = layer(x)

# Here's how to use the cell to build a stacked RNN:

cells = [MinimalRNNCell(32), MinimalRNNCell(64)]
#x = keras.Input((None, 5))

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import RNN
from keras.layers import SimpleRNN
from keras.layers import GRU
from keras.layers import SimpleRNNCell
from keras.layers import GRUCell
from keras.layers import CuDNNGRU
from keras.layers import CuDNNLSTM
from keras.layers import StackedRNNCells
from keras.layers import LSTMCell
from keras.layers import GRUCell
from keras.layers import RNN
from keras.datasets import imdb
from keras.datasets import reuters
from keras.utils import np_utils

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(RNN(cells))
model.add(Dense(46, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Using TensorFlow backend.


Loading data...
2113536/2110848 [==============================] - 0s 0us/step
8982 train sequences
2246 test sequences
Pad sequences (samples x time)
x_train shape: (8982, 80)
x_test shape: (2246, 80)
Build model...
Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 21s 2ms/step - loss: 2.5638 - acc: 0.3518 - val_loss: 2.3148 - val_acc: 0.3673
Epoch 2/3
8982/8982 [==============================] - 20s 2ms/step - loss: 2.2129 - acc: 0.3545 - val_loss: 2.3113 - val_acc: 0.3629
Epoch 3/3
2246/2246 [==============================] - 1s 385us/step
Test score: 2.366030941451116
Test accuracy: 0.36019590388210565


### SimpleRNN

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import RNN
from keras.layers import SimpleRNN
from keras.layers import GRU
from keras.layers import SimpleRNNCell
from keras.layers import GRUCell
from keras.layers import CuDNNGRU
from keras.layers import CuDNNLSTM
from keras.layers import StackedRNNCells
from keras.layers import LSTMCell
from keras.layers import GRUCell
from keras.datasets import imdb
from keras.datasets import reuters
from keras.utils import np_utils

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(46, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
8982 train sequences
2246 test sequences
Pad sequences (samples x time)
x_train shape: (8982, 80)
x_test shape: (2246, 80)
Build model...
Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 23s 3ms/step - loss: 2.6476 - acc: 0.3251 - val_loss: 2.4571 - val_acc: 0.3620
Epoch 2/3
8982/8982 [==============================] - 23s 3ms/step - loss: 2.4325 - acc: 0.3517 - val_loss: 2.4306 - val_acc: 0.3620
Epoch 3/3
2246/2246 [==============================] - 1s 481us/step
Test score: 2.4389428232571744
Test accuracy: 0.36197684778237277


### GRU

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import RNN
from keras.layers import SimpleRNN
from keras.layers import GRU
from keras.datasets import imdb
from keras.datasets import reuters
from keras.utils import np_utils

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(46, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
8982 train sequences
2246 test sequences
Pad sequences (samples x time)
x_train shape: (8982, 80)
x_test shape: (2246, 80)
Build model...
Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 43s 5ms/step - loss: 2.5616 - acc: 0.3499 - val_loss: 2.4514 - val_acc: 0.3620
Epoch 2/3
8982/8982 [==============================] - 41s 5ms/step - loss: 2.3433 - acc: 0.3544 - val_loss: 2.0543 - val_acc: 0.4016
Epoch 3/3
2246/2246 [==============================] - 2s 878us/step
Test score: 1.496661943616553
Test accuracy: 0.6219946572213755


### LSTM

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.datasets import reuters
from keras.utils import np_utils

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(46, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
8982 train sequences
2246 test sequences
Pad sequences (samples x time)
x_train shape: (8982, 80)
x_test shape: (2246, 80)
Build model...
Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 52s 6ms/step - loss: 2.5553 - acc: 0.3489 - val_loss: 2.4511 - val_acc: 0.3620
Epoch 2/3
8982/8982 [==============================] - 50s 6ms/step - loss: 2.4251 - acc: 0.3517 - val_loss: 2.4357 - val_acc: 0.3620
Epoch 3/3
2246/2246 [==============================] - 2s 1ms/step
Test score: 2.428080730956257
Test accuracy: 0.36197684778237277


### SimpleRNNCell

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import RNN
from keras.layers import SimpleRNN
from keras.layers import GRU
from keras.layers import SimpleRNNCell
from keras.datasets import imdb
from keras.datasets import reuters
from keras.utils import np_utils

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(RNN(SimpleRNNCell(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(46, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
8982 train sequences
2246 test sequences
Pad sequences (samples x time)
x_train shape: (8982, 80)
x_test shape: (2246, 80)
Build model...
Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 24s 3ms/step - loss: 2.6472 - acc: 0.3241 - val_loss: 2.4574 - val_acc: 0.3620
Epoch 2/3
8982/8982 [==============================] - 22s 3ms/step - loss: 2.4328 - acc: 0.3517 - val_loss: 2.4310 - val_acc: 0.3620
Epoch 3/3
2246/2246 [==============================] - 1s 487us/step
Test score: 2.439013553323856
Test accuracy: 0.36197684778237277


### GRUCell

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import GRU
from keras.layers import GRUCell
from keras.datasets import imdb
from keras.datasets import reuters
from keras.utils import np_utils

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(RNN(GRUCell(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(46, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
8982 train sequences
2246 test sequences
Pad sequences (samples x time)
x_train shape: (8982, 80)
x_test shape: (2246, 80)
Build model...
Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 45s 5ms/step - loss: 2.5616 - acc: 0.3499 - val_loss: 2.4514 - val_acc: 0.3620
Epoch 2/3
8982/8982 [==============================] - 43s 5ms/step - loss: 2.3433 - acc: 0.3544 - val_loss: 2.0543 - val_acc: 0.4016
Epoch 3/3
2246/2246 [==============================] - 2s 875us/step
Test score: 1.496661943616553
Test accuracy: 0.6219946572213755


### LSTMCell

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import RNN
from keras.layers import LSTMCell
from keras.datasets import imdb
from keras.datasets import reuters
from keras.utils import np_utils

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(RNN(LSTMCell(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(46, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
8982 train sequences
2246 test sequences
Pad sequences (samples x time)
x_train shape: (8982, 80)
x_test shape: (2246, 80)
Build model...
Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 54s 6ms/step - loss: 2.5553 - acc: 0.3489 - val_loss: 2.4511 - val_acc: 0.3620
Epoch 2/3
8982/8982 [==============================] - 51s 6ms/step - loss: 2.4251 - acc: 0.3517 - val_loss: 2.4357 - val_acc: 0.3620
Epoch 3/3
2246/2246 [==============================] - 2s 1ms/step
Test score: 2.428080730956257
Test accuracy: 0.36197684778237277


### StackedRNNCells

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import LSTMCell
from keras.layers import RNN
from keras.layers import StackedRNNCells
from keras.datasets import imdb
from keras.datasets import reuters
from keras.utils import np_utils

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
cells = [LSTMCell(128, dropout=0.2, recurrent_dropout=0.2),LSTMCell(128, dropout=0.3, recurrent_dropout=0.3)]
model.add(RNN(StackedRNNCells(cells)))
model.add(Dense(46, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
8982 train sequences
2246 test sequences
Pad sequences (samples x time)
x_train shape: (8982, 80)
x_test shape: (2246, 80)
Build model...
Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 93s 10ms/step - loss: 2.5418 - acc: 0.3506 - val_loss: 2.4309 - val_acc: 0.3620
Epoch 2/3
8982/8982 [==============================] - 90s 10ms/step - loss: 2.4228 - acc: 0.3517 - val_loss: 2.4350 - val_acc: 0.3620
Epoch 3/3
2246/2246 [==============================] - 4s 2ms/step
Test score: 2.4304483403930255
Test accuracy: 0.36197684778237277


### CuDNNGRU

In [1]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import RNN
from keras.layers import CuDNNGRU
from keras.datasets import imdb
from keras.datasets import reuters
from keras.utils import np_utils

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)
# 他クラス分類対応
y_train = np_utils.to_categorical(y_train)
# 他クラス分類対応
y_test = np_utils.to_categorical(y_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(CuDNNGRU(128))
model.add(Dense(46, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Using TensorFlow backend.


Loading data...
2113536/2110848 [==============================] - 1s 1us/step
8982 train sequences
2246 test sequences
Pad sequences (samples x time)
x_train shape: (8982, 80)
x_test shape: (2246, 80)
Build model...
Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 8s 929us/step - loss: 2.5501 - acc: 0.2161 - val_loss: 2.4057 - val_acc: 0.2110
Epoch 2/3
8982/8982 [==============================] - 6s 699us/step - loss: 2.1100 - acc: 0.3378 - val_loss: 1.9679 - val_acc: 0.4123
Epoch 3/3
2246/2246 [==============================] - 0s 199us/step
Test score: 1.5760941053222357
Test accuracy: 0.5939447907656299


### CuDNNLSTM

In [2]:
# 他クラス分類対応
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import RNN
from keras.layers import SimpleRNN
from keras.layers import GRU
from keras.layers import SimpleRNNCell
from keras.layers import GRUCell
from keras.layers import CuDNNGRU
from keras.layers import CuDNNLSTM
from keras.datasets import imdb
from keras.datasets import reuters
from keras.utils import np_utils

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)
# 他クラス分類対応
y_train = np_utils.to_categorical(y_train)
# 他クラス分類対応
y_test = np_utils.to_categorical(y_test)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(CuDNNLSTM(128))
# 他クラス分類対応
model.add(Dense(46, activation='sigmoid'))

# try using different optimizers and different optimizer configs
# 他クラス分類対応
model.compile(loss='categorical_crossentropy',
#model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
8982 train sequences
2246 test sequences
Pad sequences (samples x time)
x_train shape: (8982, 80)
x_test shape: (2246, 80)
Build model...
Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 7s 755us/step - loss: 2.5519 - acc: 0.3515 - val_loss: 2.4473 - val_acc: 0.3620
Epoch 2/3
8982/8982 [==============================] - 6s 678us/step - loss: 2.4303 - acc: 0.3517 - val_loss: 2.4327 - val_acc: 0.3620
Epoch 3/3
2246/2246 [==============================] - 0s 192us/step
Test score: 2.426268374399744
Test accuracy: 0.36197684778237277


## モデルの結果を比較
|Model |Test score  |Test accuracy  |
|---|---|---|
|RNN |2.366 |0.3601 |
|SimpleRNN |2.438 |0.3619 |
|GRU |1.496 |0.6219 |
|LSTM |2.428 |0.3619 |
|SimpleRNNCell |2.439 |0.3619 |
|GRUCell |1.496 |0.6219 |
|LSTMCell |2.428 |0.3619 |
|CuDNNGRU |1.576 |0.5939 |
|CuDNNLSTM |2.426 |0.3619 |


ロイターのデータセットをepoch３回に限定した結果でみると、GRU、GRUCell、次にCuDNNGRUと、GRU系の精度が最も良かった。